In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import time
import os 
content_dir = os.getcwd() + '/test_content/'
style_dir = os.getcwd() + '/test_style/'
model_dir = os.getcwd() + '/Models/'

import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import torchvision
from torchvision import transforms

from PIL import Image
from collections import OrderedDict

device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
#vgg definition that conveniently let's you grab the outputs from any layer
class VGG(nn.Module):
    def __init__(self, pool='max'):
        super(VGG, self).__init__()
        #vgg modules
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv3_4 = nn.Conv2d(256, 256, kernel_size=3, padding=1)
        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv4_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        self.conv5_4 = nn.Conv2d(512, 512, kernel_size=3, padding=1)
        if pool == 'max':
            self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pool == 'avg':
            self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool3 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool4 = nn.AvgPool2d(kernel_size=2, stride=2)
            self.pool5 = nn.AvgPool2d(kernel_size=2, stride=2)
    def forward(self, x, out_keys):
        out = {}
        out['r11'] = F.relu(self.conv1_1(x))
        out['r12'] = F.relu(self.conv1_2(out['r11']))
        out['p1'] = self.pool1(out['r12'])
        out['r21'] = F.relu(self.conv2_1(out['p1']))
        out['r22'] = F.relu(self.conv2_2(out['r21']))
        out['p2'] = self.pool2(out['r22'])
        out['r31'] = F.relu(self.conv3_1(out['p2']))
        out['r32'] = F.relu(self.conv3_2(out['r31']))
        out['r33'] = F.relu(self.conv3_3(out['r32']))
        out['r34'] = F.relu(self.conv3_4(out['r33']))
        out['p3'] = self.pool3(out['r34'])
        out['r41'] = F.relu(self.conv4_1(out['p3']))
        out['r42'] = F.relu(self.conv4_2(out['r41']))
        out['r43'] = F.relu(self.conv4_3(out['r42']))
        out['r44'] = F.relu(self.conv4_4(out['r43']))
        out['p4'] = self.pool4(out['r44'])
        out['r51'] = F.relu(self.conv5_1(out['p4']))
        out['r52'] = F.relu(self.conv5_2(out['r51']))
        out['r53'] = F.relu(self.conv5_3(out['r52']))
        out['r54'] = F.relu(self.conv5_4(out['r53']))
        out['p5'] = self.pool5(out['r54'])
        return [out[key] for key in out_keys]

In [3]:
# gram matrix and loss
class GramMatrix(nn.Module):
    def forward(self, input):
        b,c,h,w = input.size()
        F = input.view(b, c, h*w)
        G = torch.bmm(F, F.transpose(1,2)) 
        G.div_(h*w)
        return G

class GramMSELoss(nn.Module):
    def forward(self, input, target):
        out = nn.MSELoss()(GramMatrix()(input), target)
        return(out)

In [4]:
from transformers import pipeline


checkpoint = "vinvino02/glpn-nyu"
depth_estimator = pipeline("depth-estimation", model=checkpoint,device=device)

def depth_loss(synthesis_image,content_image,depth_estimator):
    if True:
        tensor = synthesis_image
        tensor = tensor.cpu().clone()
        tensor = tensor.squeeze(0)
        tensor = tensor.permute(1, 2, 0)
        image = tensor.detach().numpy()
        image = (image * 255).astype(np.uint8)
        synthesis_image = Image.fromarray(image)
    image_size = synthesis_image.size
    synthesis_image = synthesis_image.resize((image_size[0]//3,image_size[1]//3))
    content_image = content_image.resize((image_size[0]//3,image_size[1]//3))
    synthesis_depth = depth_estimator(synthesis_image)["depth"]
    synthesis_depth = torch.from_numpy(np.asarray(synthesis_depth).copy()).float()
    synthesis_depth = (synthesis_depth-torch.min(synthesis_depth))/(torch.max(synthesis_depth)-torch.min(synthesis_depth))
    content_depth = depth_estimator(content_image)["depth"]
    content_depth = torch.from_numpy(np.asarray(content_depth).copy()).float()
    content_depth = (content_depth-torch.min(content_depth))/(torch.max(content_depth)-torch.min(content_depth))
    return torch.sum(torch.abs(synthesis_depth-content_depth))
    

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.


In [ ]:
# pre and post processing for images
img_size = 512 
prep = transforms.Compose([transforms.Resize(img_size),
                           transforms.ToTensor(),
                           transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to BGR
                           transforms.Normalize(mean=[0.40760392, 0.45795686, 0.48501961], #subtract imagenet mean
                                                std=[1,1,1]),
                           transforms.Lambda(lambda x: x.mul_(255)),
                          ])
postpa = transforms.Compose([transforms.Lambda(lambda x: x.mul_(1./255)),
                           transforms.Normalize(mean=[-0.40760392, -0.45795686, -0.48501961], #add imagenet mean
                                                std=[1,1,1]),
                           transforms.Lambda(lambda x: x[torch.LongTensor([2,1,0])]), #turn to RGB
                           ])
postpb = transforms.Compose([transforms.ToPILImage()])
def postp(tensor): # to clip results in the range [0,1]
    t = postpa(tensor)
    t[t>1] = 1    
    t[t<0] = 0
    img = postpb(t)
    return img

In [ ]:
#get network
vgg = VGG()
vgg.load_state_dict(torch.load(model_dir + 'vgg_conv.pth'))
#from torchvision import transforms, models
#vgg = models.vgg19(pretrained=True)
for param in vgg.parameters():
    param.requires_grad = False
if torch.cuda.is_available():
    vgg.cuda()

lap = LaplacianNetwork()
for param in lap.parameters():
    param.requires_grad = False
if torch.cuda.is_available():
    lap.cuda()

In [9]:
#load images, ordered as [style_image, content_image]
img_dirs = [style_dir,content_dir]
content_names = [f for f in os.listdir(content_dir) if os.path.isfile(os.path.join(content_dir, f))]
style_names = [f for f in os.listdir(style_dir) if os.path.isfile(os.path.join(style_dir, f))]
print(content_names)
print(style_names)
for cn in content_names:
    for sn in style_names:
        img_names = [sn,cn]
        imgs = [Image.open(img_dirs[i] + name) for i,name in enumerate(img_names)]
        imgs_torch = [prep(img) for img in imgs]
        if torch.cuda.is_available():
            imgs_torch = [Variable(img.unsqueeze(0).cuda()) for img in imgs_torch]
        else:
            imgs_torch = [Variable(img.unsqueeze(0)) for img in imgs_torch]
        style_image, content_image = imgs_torch

        opt_img = Variable(torch.randn(content_image.size()).type_as(content_image.data), requires_grad=True) #random init
        #opt_img = Variable(content_image.data.clone(), requires_grad=True)
        for img in imgs:
            plt.imshow(img);plt.show()
        #define layers, loss functions, weights and compute optimization targets
        style_layers = ['r11','r21','r31','r41', 'r51'] 
        content_layers = ['r42']
        loss_layers = style_layers + content_layers
        loss_fns = [GramMSELoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)
        if torch.cuda.is_available():
            loss_fns = [loss_fn.cuda() for loss_fn in loss_fns]
    
        #these are good weights settings:
        style_weights = [1e3/n**2 for n in [64,128,256,512,512]]
        content_weights = [1e0]
        weights = style_weights + content_weights
        
        #compute optimization targets
        style_targets = [GramMatrix()(A).detach() for A in vgg(style_image, style_layers)]
        content_targets = [A.detach() for A in vgg(content_image, content_layers)]
        targets = style_targets + content_targets

        #run style transfer
        max_iter = 400
        show_iter = 50
        optimizer = optim.LBFGS([opt_img],lr=1);
        optimizer = optimizer
        n_iter=[0]

        while n_iter[0] <= max_iter:
            torch.cuda.empty_cache()
            def closure():

                optimizer.zero_grad()
                out = vgg(opt_img, loss_layers)
                layer_losses = [weights[a] * loss_fns[a](A, targets[a]) for a,A in enumerate(out)]
                deep_loss = depth_loss(opt_img,imgs[1],depth_estimator=depth_estimator)
                loss = torch.sum(torch.stack(layer_losses)) + deep_loss 
                loss.backward()
                n_iter[0]+=1
                if n_iter[0]%show_iter == (show_iter-1):
                    print('Iteration: %d, loss: %f'%(n_iter[0]+1, loss.item()))
                return loss
            optimizer.step(closure)
            if n_iter[0] >= max_iter:
                break
        #display result
        out_img = postp(opt_img.data[0].cpu().squeeze())
        plt.imshow(out_img)
        plt.show()
        plt.imsave("Depth"+cn+sn+".png",out_img)
        gcf().set_size_inches(10,10)